In [3]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# (201105) 추가한 모듈
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# pandas
import pandas as pd

In [4]:
def get_youtube_main_video_all_data(url):
    
    # webdriver 초기화
    driver_path ='./chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    time.sleep(5)
    
    thumbnail_list = [] # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = [] # 썸네일 제목을 저장하는 리스트
    overlay_time_list = [] # 동영상 재생시간들을 저장하는 리스트
    channel_profile_list = [] # 채널 프로필사진 주소 저장용 리스트
    channel_name_list = [] # 채널 이름 저장용 리스트
    hits_str_list = [] # 조회수 문자열 저장용 리스트
    update_date_list = [] # 업로드 시간 저장용 리스트
    
    # 첫 동영상 시작 시 시작번호(index = 1)
    index = 1
    # 한 번 스크롤하고 멈춤 시간 설정
    SCROLL_PAUSE_TIME = 2
    
    # num_of_pagedowns = 10
    
    while True:
        try:
            thumbnail_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a/yt-formatted-string'
            overlay_time_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/div[1]/ytd-thumbnail-overlay-time-status-renderer/span'
            channel_profile_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/a/yt-img-shadow/img'
            channel_name_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string/a'
            hits_str_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[1]'
            update_date_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[2]'
            # 동영상 데이터 링크의 실제 url
            
            # 이미지, 채널 프로필, 오버레이 타임이 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            thumb = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, thumbnail_xpath)))
            overlay = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, overlay_time_xpath)))
            ch_profile = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, channel_profile_xpath)))

            if (thumb or overlay or ch_profile) is None:
                print("img or overlay or ch_profile is not loaded.")

            # 동영상 목록을 추가 불러오기 위해 스크롤 내림(pass)
            # index % 8 일 경우 스크롤을 내림(8개씩 다 볼 경우) -> page_down key로 내린다
            if index % 8 == 0:
                # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                
            # 썸네일 주소를 리스트에 저장
            thumbnail = driver.find_element_by_xpath(thumbnail_xpath)
            thumbnail_url = thumbnail.get_attribute('src')
            thumbnail_list.append(thumbnail_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            title_list.append(title.text)

            # 동영상 재생시간을 리스트에 저장
            overlay_time = driver.find_element_by_xpath(overlay_time_xpath)
            overlay_time_list.append(overlay_time.text)

            # 채널 프로필 사진 링크를 리스트에 저장
            channel_profile = driver.find_element_by_xpath(channel_profile_xpath)
            channel_profile_url = channel_profile.get_attribute('src')
            channel_profile_list.append(channel_profile_url)

            # 채널이름을 리스트에 저장
            channel_name = driver.find_element_by_xpath(channel_name_xpath)
            channel_name_list.append(channel_name.text)

            # 조회수 문자열을 리스트에 저장
            hits_str = driver.find_element_by_xpath(hits_str_xpath)
            hits_str_list.append(hits_str.text)

            # 업데이트 시간을 리스트에 저장
            update_date = driver.find_element_by_xpath(update_date_xpath)
            update_date_list.append(update_date.text)

            # 리스트에 들어간 데이터 중간 확인
            print('index_no: ', index)
            print('title_text:', title.text, ' img_url: ', thumbnail_url)
            print('overlay_time: ', overlay_time.text, ' channel_profile: ', channel_profile_url)
            print('channel_name: ', channel_name.text, ' hits_str: ', hits_str.text, ' update_date: ', update_date.text) 
            
            index += 1 # 다음 영상데이터로 넘어간다
        
        except Exception as e:
            print()
            print("Exception: ", e)
            break
    
    driver.close()
    
    # pandas DataFrame 형태로 데이터를 테이블에 담아 리턴
    for thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date in zip(thumbnail_list, title_list, overlay_time_list, channel_profile_list, channel_name_list,
        hits_str_list, update_date_list):
        print(thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date)
    
    # making dataframe
    dic = {'thumbnail': thumbnail_list, 'title': title_list, 'overlay_time': overlay_time_list, 'channel_profile': channel_profile_list,
          'channel_name': channel_name_list, 'hits_str': hits_str_list, 'update_date': update_date_list}
    
    df = pd.DataFrame(dic)
    
    return df

In [5]:
test_url = 'https://www.youtube.com/'
get_youtube_main_video_all_data(test_url)


Exception:  Message: 



,thumbnail,title,overlay_time,channel_profile,channel_name,hits_str,update_date
